# Quora Question Pair - Train Model with RNN + Attension + LSTM

## Load Training Set

In [1]:
from __future__ import print_function
%matplotlib inline
import numpy as np
import pandas as pd
import datetime, time, json
from keras.models import Model, Sequential
from keras.layers import Input, TimeDistributed, Dense, Lambda, concatenate, Dropout, BatchNormalization
from keras.layers.embeddings import Embedding
from keras.regularizers import l2
from keras.callbacks import Callback, ModelCheckpoint
from keras import backend as K
from sklearn.model_selection import train_test_split

from importlib import reload
import dev_layers

Using TensorFlow backend.


In [14]:
# const
MAX_SEQUENCE_LENGTH = 30
MAX_NB_WORDS = 200000
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.1
TEST_SPLIT = 0.1
RNG_SEED = 13371447
NB_EPOCHS = 5
DROPOUT = 0.1
BATCH_SIZE = 256
L2_WEIGHT_DECAY = 0
# files
Q1_TRAINING_DATA_FILE = 'q1_train.npy'
Q2_TRAINING_DATA_FILE = 'q2_train.npy'
Q1_TEST_DATA_FILE = 'q1_test.npy'
Q2_TEST_DATA_FILE = 'q2_test.npy'
TEST_ID_FILE = 'test_ids.npy'
LABEL_TRAINING_DATA_FILE = 'label_train.npy'
WORD_EMBEDDING_MATRIX_FILE = 'word_embedding_matrix.npy'
NB_WORDS_DATA_FILE = 'nb_words.json'
# save params in learning
MODEL_WEIGHTS_FILE = 'question_pairs_weights.h5'

In [3]:
q1_data = np.load(open(Q1_TRAINING_DATA_FILE, 'rb'))
q2_data = np.load(open(Q2_TRAINING_DATA_FILE, 'rb'))
labels = np.load(open(LABEL_TRAINING_DATA_FILE, 'rb'))
word_embedding_matrix = np.load(open(WORD_EMBEDDING_MATRIX_FILE, 'rb'))
with open(NB_WORDS_DATA_FILE, 'r') as f:
    nb_words = json.load(f)['nb_words']

In [4]:
print('Shape of question1 data tensor:', q1_data.shape)
print('Shape of question2 data tensor:', q2_data.shape)
print('Shape of label tensor:', labels.shape)

Shape of question1 data tensor: (404290, 30)
Shape of question2 data tensor: (404290, 30)
Shape of label tensor: (404290,)


In [5]:
X = np.stack((q1_data, q2_data), axis=1)
y = labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SPLIT, random_state=RNG_SEED)
Q1_train = X_train[:,0]
Q2_train = X_train[:,1]
Q1_test = X_test[:,0]
Q2_test = X_test[:,1]

## Model

In [15]:
reload(dev_layers)

<module 'dev_layers' from 'd:\\Mycode\\python\\ECE449Project\\dev_layers.py'>

In [16]:
question1 = Input(shape=(MAX_SEQUENCE_LENGTH,))
question2 = Input(shape=(MAX_SEQUENCE_LENGTH,))

q1 = Embedding(nb_words + 1, 
                 EMBEDDING_DIM, 
                 weights=[word_embedding_matrix], 
                 input_length=MAX_SEQUENCE_LENGTH, 
                 trainable=False)(question1)
q1 = TimeDistributed(Dense(EMBEDDING_DIM, activation='relu'))(q1)

q2 = Embedding(nb_words + 1, 
                 EMBEDDING_DIM, 
                 weights=[word_embedding_matrix], 
                 input_length=MAX_SEQUENCE_LENGTH, 
                 trainable=False)(question2)
q2 = TimeDistributed(Dense(EMBEDDING_DIM, activation='relu'))(q2)

q1 = dev_layers.BiLSTM_Layer(MAX_SEQUENCE_LENGTH, EMBEDDING_DIM)(q1)
q2 = dev_layers.BiLSTM_Layer(MAX_SEQUENCE_LENGTH, EMBEDDING_DIM)(q2)

q1, q2 = dev_layers.Attention_Layer()(q1, q2)

q1 = dev_layers.BiLSTM_Layer(MAX_SEQUENCE_LENGTH, EMBEDDING_DIM, 4)(q1)
q2 = dev_layers.BiLSTM_Layer(MAX_SEQUENCE_LENGTH, EMBEDDING_DIM, 4)(q2)

merged = dev_layers.Pooling_Layer(EMBEDDING_DIM, 600, 4, DROPOUT, l2_weight_decay=L2_WEIGHT_DECAY)(q1, q2)

merged = Dense(300, activation='relu')(merged)
merged = Dropout(DROPOUT)(merged)
merged = BatchNormalization()(merged)

merged = Dense(100, activation='relu')(merged)
merged = Dropout(DROPOUT)(merged)
merged = BatchNormalization()(merged)

merged = Dense(10, activation='relu')(merged)
merged = Dropout(DROPOUT)(merged)
merged = BatchNormalization()(merged)

is_duplicate = Dense(1, activation='sigmoid')(merged)

model = Model(inputs=[question1,question2], outputs=is_duplicate)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 30)           0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 30)           0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 30, 300)      36150000    input_3[0][0]                    
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 30, 300)      36150000    input_4[0][0]                    
__________________________________________________________________________________________________
time_distr

In [12]:
print("Starting training at", datetime.datetime.now())
t0 = time.time()
# callbacks = [ModelCheckpoint(MODEL_WEIGHTS_FILE, monitor='val_accuracy', save_best_only=True)]
callbacks = [ModelCheckpoint(MODEL_WEIGHTS_FILE, monitor='val_acc', save_best_only=True)]
history = model.fit([Q1_train, Q2_train],
                    y_train,
                    epochs=NB_EPOCHS,
                    validation_split=VALIDATION_SPLIT,
                    verbose=2,
                    batch_size=BATCH_SIZE,
                    callbacks=callbacks)
t1 = time.time()
print("Training ended at", datetime.datetime.now())
print("Minutes elapsed: %f" % ((t1 - t0) / 60.))

Starting training at 2021-05-16 03:53:47.733196
Train on 327474 samples, validate on 36387 samples
Epoch 1/25
 - 1275s - loss: 0.5448 - acc: 0.7235 - val_loss: 0.4994 - val_acc: 0.7568
Epoch 2/25
 - 1232s - loss: 0.5034 - acc: 0.7503 - val_loss: 0.4962 - val_acc: 0.7604
Epoch 3/25
 - 1217s - loss: 0.4894 - acc: 0.7594 - val_loss: 0.4823 - val_acc: 0.7666
Epoch 4/25
 - 1196s - loss: 0.4784 - acc: 0.7658 - val_loss: 0.4714 - val_acc: 0.7668
Epoch 5/25
 - 1196s - loss: 0.4709 - acc: 0.7704 - val_loss: 0.4752 - val_acc: 0.7726
Epoch 6/25
 - 1194s - loss: 0.4702 - acc: 0.7709 - val_loss: 0.4772 - val_acc: 0.7659
Epoch 7/25
 - 1190s - loss: 0.4677 - acc: 0.7730 - val_loss: 0.4708 - val_acc: 0.7700
Epoch 8/25
 - 1190s - loss: 0.4657 - acc: 0.7734 - val_loss: 0.4824 - val_acc: 0.7707
Epoch 9/25
 - 1190s - loss: 0.4659 - acc: 0.7742 - val_loss: 0.4711 - val_acc: 0.7744
Epoch 10/25
 - 1190s - loss: 0.4602 - acc: 0.7763 - val_loss: 0.4704 - val_acc: 0.7702
Epoch 11/25
 - 1186s - loss: 0.4665 - ac

KeyboardInterrupt: 

In [13]:
acc = pd.DataFrame({'epoch': [ i + 1 for i in history.epoch ],
                    'training': history.history['acc'],
                    'validation': history.history['val_acc']})
print(acc)

NameError: name 'history' is not defined

In [12]:
max_val_acc, idx = max((val, idx) for (idx, val) in enumerate(history.history['val_acc']))
print('Maximum accuracy at epoch', '{:d}'.format(idx+1), '=', '{:.4f}'.format(max_val_acc))

Maximum accuracy at epoch 5 = 0.7748


## Test Performance 

In [13]:
model.load_weights(MODEL_WEIGHTS_FILE)
loss, accuracy = model.evaluate([Q1_test, Q2_test], y_test, verbose=0)
print('loss = {0:.4f}, accuracy = {1:.4f}'.format(loss, accuracy))

loss = 0.4641, accuracy = 0.7773


## Load Test Set

In [ ]:
# q1_test_data = np.load(open(Q1_TEST_DATA_FILE, 'rb'))
# q2_test_data = np.load(open(Q2_TEST_DATA_FILE, 'rb'))
# test_ids = np.load(open(TEST_ID_FILE, 'rb'))
# print('Shape of question1 test data tensor:', q1_test_data.shape)
# print('Shape of question2 test data tensor:', q2_test_data.shape)
# print('Shape of label tensor:', test_ids.shape)